We test the SqLinOp wrapper for computing eigenvalues of implicitly represented square linear operators.

It seems that the power method code eigvals_power from the *IterativeSolvers* package is broken?

But eigvals_lanczos might be working, however it's not clear how to get only the largest eig, since a call for one eig seems to sometime produce a smaller eig?

It's probably better just to go for the *eigs* function from standard julia lin alg.

*Tentatively* it seems that using :LM option for eigs gives the eigenvalue that's largest in magnitude, which is good enough for our purposes?

Testing with our LDL
--------------------

In [1]:
using Laplacians

In [2]:
include("../src/samplingSolver.jl")

purge (generic function with 1 method)

In [3]:
A = chimera(10,1);

In [4]:
n = A.n

10

In [13]:
f,gOp,U,d = buildSolver(A);

In [6]:
fOp = SqLinOp(true,1.0,n,f)

SqLinOp{Float64,Int64}(true,1.0,10,(anonymous function))

In [8]:
@time eigs(fOp;nev=1,which=:LM)[1][1]

  0.000609 seconds (491 allocations: 55.266 KB)


2.1604506204523295

In [9]:
L = lap(A);

In [10]:
norm(Symmetric(L)-L)

0.0

In [11]:
eigs(Symmetric(pinv(full(L)));nev=1,which=:LM)[1][1]

2.1888891924987695

looks reasonable?

In [15]:
@time eigs(gOp;nev=1,which=:LM)[1][1]

  0.000626 seconds (781 allocations: 59.797 KB)


-0.04724310116513397

In [16]:
@time eigs(gOp)[1]

  0.002700 seconds (1.37 k allocations: 93.650 KB)


6-element Array{Float64,1}:
 -0.0472431 
  0.0280675 
 -0.00772168
  0.00672949
  0.00456993
  6.61894e-5

In [17]:
norm(full(L - U * spdiagm(d) * U'))

0.09518690709748781

In [18]:
dih = zeros(n); dih[1:(n-1)] = d[1:(n-1)].^(-1/2); dih'

1x10 Array{Float64,2}:
 0.57735  0.609961  0.617282  0.707107  …  0.887277  0.965056  1.1522  0.0

In [19]:
Ui = full(U)^(-1);

In [20]:
Papx = diagm(dih)*Ui*L*Ui'*diagm(dih);

In [21]:
P = eye(n); P[n,n] = 0;

In [23]:
@time eigs(Symmetric(Papx-P);nev=1,which=:LM)[1][1]

  0.016323 seconds (5.19 k allocations: 275.069 KB, 35.04% gc time)


-0.04724310116513395

> eigs(gOp;nev=1,which=:LM)[1][1]

agrees with

> eigs(Symmetric(Papx-P);nev=1,which=:LM)[1][1]

so I think the implicit representation is working.

How to do the check
-------------------

In [3]:
A = chimera(700,1); n = A.n

700

In [ ]:
@time for i in 1:10
    f,gOp,U,d = buildSolver(A);
end

In [14]:
abs(checkError(gOp)) < 0.5

true

In [15]:
checkError(gOp)

0.04387413598875789

In [74]:
Lapx = U * spdiagm(d) * U';

In [77]:
Lapxih = real(pinv(full(Lapx))^(1/2))

200x200 Array{Float64,2}:
  0.360271     -0.00588109   -0.000990261  …  -0.00524851   -0.00799241
 -0.00588109    0.445984      0.000636373      0.00220823   -0.00638217
 -0.000990261   0.000636373   0.564645        -0.00375238   -0.00892213
 -0.00797588   -0.0059753    -0.00918831      -0.00760754    0.108593  
  0.0373765    -0.00671127   -0.00582073      -0.00570369   -0.00771565
  0.0235354    -0.00628711   -0.00558339   …  -0.00193305   -0.00757222
 -0.00643732   -0.0053051    -0.00741875      -0.0061257     0.0654419 
 -0.000195251  -0.00611914    0.00288022      -0.0068299    -0.00777341
 -0.00530052   -0.0065231    -0.00676912      -0.00512698   -0.0081187 
  0.000983122   0.00186865   -0.00650931      -0.00338515   -0.00538942
  0.00581447   -0.00747657   -0.00554944   …  -0.00399302   -0.00827748
 -0.00751793   -0.00441592   -0.00878342      -0.00735153    0.0509908 
 -0.000925103  -0.00708319   -0.00536065       0.00511356   -0.00822426
  ⋮                                   

In [80]:
norm(Lapxih*lap(A)*Lapxih - (eye(n)-ones(n,n)/n))

0.03085431012087925